## Готовим данные для модели стратегического прогноза

In [1]:
%pylab inline
%matplotlib inline

import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import MySQLdb

from datetime import timedelta
import datetime

import glob

Populating the interactive namespace from numpy and matplotlib


**Загрузка данных из базы**

In [ ]:
# %%time
# db_connection = MySQLdb.connect(host="194.247.15.117", 
#                      port=7806,
#                      user="bar",
#                      passwd="PASSWORD",
#                      db="Inventory")

# data = pd.read_sql('SELECT * FROM Inventory.vTotalPassengers_v2', con=db_connection)
# db_connection.close()

# print(len(data))

**Загрузка данных с локального хранилища**

In [ ]:
# data.to_csv('data.csv', sep=';')

In [2]:
data = pd.read_csv('../data/data.csv', sep=';')
data.drop('Unnamed: 0', inplace=True, axis = 1)

**Смотрим на данные**

In [ ]:
bdata = data.copy()

In [ ]:
# сортируем по рейсу, дате
data.sort_values(by=['FlightID', 'DptDt'], inplace=True)

# удалим неполные прогнозы
data = data[data['reportDt'] >= data['fltNrDptDt']]

In [ ]:
data.info()

In [ ]:
data['resHoldTotal'].hist()

In [ ]:
data

In [ ]:
# 3 сигмы для поездов 
df = data.groupby(['FlightID'], as_index = False)['resHoldTotal'].sum()
sigma3 = np.mean(df['resHoldTotal']) + 3 * np.std(df['resHoldTotal'])
sigma2m = np.mean(df['resHoldTotal']) - 2 * np.std(df['resHoldTotal'])

data[~data.FlightID.isin(df[(df.resHoldTotal > sigma3) | (df.resHoldTotal < sigma3m)].FlightID.values)]

In [ ]:
np.mean(df['resHoldTotal']), np.std(df['resHoldTotal'])

In [ ]:
np.mean(df['resHoldTotal']) - np.std(df['resHoldTotal']), np.mean(df['resHoldTotal']) + 2 * np.std(df['resHoldTotal'])

In [ ]:
data['resHoldTotal'].value_counts()

In [ ]:
data.groupby(['FlightID', 'reportDt', 'fltNrDptDt'], as_index = False)['resHoldTotal', 'seatsAvail', 'authLevel'].sum().sort_values(['resHoldTotal', 'FlightID']).head(50)

In [ ]:
data[data.FlightID.isin(df[df['resHoldTotal'] == 0]['FlightID'].unique())].sort_values(['FlightID', 'DptDt'])#.to_csv('flights_with_zero_ResHoldTotal.csv', sep=';')

**Предобработка данных**

In [ ]:
# номер недели
def week_of_month(dt):
    first_day = dt.replace(day=1)
    dom = dt.day
    adjusted_dom = dom + first_day.weekday()
    return int(ceil(adjusted_dom/7.0))

In [ ]:
segment_dict= {}
flightNrSf_dict= {}
code_dict= {}
segment_dict2= {}

def prepareData(data):
    # сортируем по рейсу, дате
    data.sort_values(by=['FlightID', 'DptDt'], inplace=True)

    # удалим неполные прогнозы
    data = data[data['reportDt'] >= data['fltNrDptDt']]

    # drop stuff
    data.drop('reportDt', axis = 1, inplace = True)
    data.drop('seatsAvail', axis = 1, inplace = True)
    data.drop('authLevel', axis = 1, inplace = True)
    data.drop('Carrier', axis = 1, inplace = True)
    
    # make flight id 
    data['flightNrSf'] = [str(data.fltNr.values[x]) + data.opsSuffix.values[x] for x in range(0, len(data.fltNr.values))]
    data.drop('fltNr', axis = 1, inplace = True)
    data.drop('opsSuffix', axis = 1, inplace = True)
    
    # словари для кодирования позедов, сегментов и классов вагонов
    segment_dict = dict(enumerate(set(np.append(data.orig.unique(), data.dest.unique()))))
    flightNrSf_dict = dict(enumerate(data.flightNrSf.unique()))
    code_dict = dict(enumerate(data.Code.unique()))

    # кодируем сегмент
    data.orig = data.orig.map(lambda x: list(segment_dict.keys())[list(segment_dict.values()).index(x)])
    data.dest = data.dest.map(lambda x: list(segment_dict.keys())[list(segment_dict.values()).index(x)])

    # кодируем поезда
    data.flightNrSf = data.flightNrSf.map(lambda x: list(flightNrSf_dict.keys())[list(flightNrSf_dict.values()).index(x)])

    # кодируем вагоны
    data.Code = data.Code.map(lambda x: list(code_dict.keys())[list(code_dict.values()).index(x)])

    # make segment
    data['segment'] = data.orig.astype('str') + '_' + data.dest.astype('str')
    data.drop('orig', axis = 1, inplace = True)
    data.drop('dest', axis = 1, inplace = True)

    # словарь для повторного кодирования сегментов
    segment_sec_dict2 = dict(enumerate(data.segment.unique()))

    # повторно кодируем сегмент
    data.segment = data.segment.map(lambda x: list(segment_sec_dict.keys())[list(segment_sec_dict.values()).index(x)])

    # кодируем дату
    data.DptDt = pd.DatetimeIndex(data.DptDt)
    data.fltNrDptDt = pd.DatetimeIndex(data.fltNrDptDt)
    data.fltNrArrDt = pd.DatetimeIndex(data.fltNrArrDt)

    data.sort_values(['FlightID', 'DptDt'], inplace=True)
    data.reset_index(inplace=True)
    data.drop('index', axis = 1, inplace = True)

    data['DayOfWeekDpt'] = pd.DatetimeIndex(data['DptDt']).dayofweek 
    data['WeekOfMonthDpt'] = [week_of_month(x) for x in pd.DatetimeIndex(data['DptDt'])]
    data['MonthDpt'] = pd.DatetimeIndex(data['DptDt']).month 
    data['QuarterDpt'] = pd.DatetimeIndex(data['DptDt']).quarter 
    data['YearDpt'] = pd.DatetimeIndex(data['DptDt']).year
    data['IsWeekendDpt'] = data['DayOfWeekDpt'].isin([5,6])*1

    data['DayOfWeekArr'] = pd.DatetimeIndex(data['fltNrArrDt']).dayofweek 
    data['WeekOfMonthArr'] = [week_of_month(x) for x in pd.DatetimeIndex(data['DptDt'])]
    data['MonthArr'] = pd.DatetimeIndex(data['DptDt']).month 
    data['QuarterArr'] = pd.DatetimeIndex(data['DptDt']).quarter 
    data['YearArr'] = pd.DatetimeIndex(data['DptDt']).year
    data['IsWeekendArr'] = data['DayOfWeekArr'].isin([5,6])*1

    # праздники
    data['Holiday'] = 0
    data['IsHoliday'] = 0
    holidayDays = ('2017-03-08 00:00:00', '2017-02-23 00:00:00', '2017-02-14 00:00:00', '2017-01-07 00:00:00', '2017-06-12 00:00:00', '2017-01-01 00:00:00', '2017-05-09 00:00:00',
                   '2016-03-08 00:00:00', '2016-02-23 00:00:00', '2016-02-14 00:00:00', '2016-01-07 00:00:00', '2016-06-12 00:00:00', '2016-01-01 00:00:00', '2016-05-09 00:00:00',
                   '2015-03-08 00:00:00', '2015-02-23 00:00:00', '2015-02-14 00:00:00', '2015-01-07 00:00:00', '2015-06-12 00:00:00', '2015-01-01 00:00:00', '2015-05-09 00:00:00',
                   '2014-03-08 00:00:00', '2014-02-23 00:00:00', '2014-02-14 00:00:00', '2014-01-07 00:00:00', '2014-06-12 00:00:00', '2014-01-01 00:00:00', '2014-05-09 00:00:00')

    for holidayIndex in holidayDays:
        daysIncr = np.arange(-3, 3)
        holidayCode = np.append(np.arange(1, 4), np.arange(1, 4)[::-1])

        for i, dayIncr in enumerate(daysIncr):
            holiday = pd.DatetimeIndex([holidayIndex])[0] + datetime.timedelta(days = int(dayIncr))
            data.loc[data[data.DptDt == holiday].index.values, 'Holiday'] = holidayCode[i]

            if dayIncr == 0:
                data.loc[data[data.DptDt == holiday].index.values, 'IsHoliday'] = 1
                
    return data

In [ ]:
data = prepareData(data)

In [ ]:
data['Lag'] = pd.DatetimeIndex(data['fltNrDptDt']).

In [ ]:
lags = list(1, 2, 4, 8, 16, 52)
for lag in lags:
    data['Lag_' + str(lag)] = data[data['Дата'] > data['Дата'] - data['Lag'] - datetime.timedelta(days = lag * 7)]['resHoldTotal'].sum()

In [ ]:
print(flightNrSf_dict)

In [ ]:
print(code_dict)

In [ ]:
print(segment_dict)

In [ ]:
print(segment_sec_dict2)

In [ ]:
data = pd.to_csv('../data/prepared_data.csv', sep=';')

**Общий прогзноз + год вперед**

In [ ]:
wdata = bdata.copy()

In [ ]:
# дополним год
df = wdata[wdata['DptDt'] > wdata['DptDt'].max() - datetime.timedelta(days=365)]
df['DptDt'] = df['DptDt'].map(lambda x: x + datetime.timedelta(days=365))
df['reportDt'] = df['reportDt'].map(lambda x: x + datetime.timedelta(days=365))
df['fltNrDptDt'] = df['fltNrDptDt'].map(lambda x: x + datetime.timedelta(days=365))
df['fltNrArrDt'] = df['fltNrArrDt'].map(lambda x: x + datetime.timedelta(days=365))

In [ ]:
wdata = wdata.append(df)

In [ ]:
wdata = prepareData(wdata)
wdata.sort_values('FlightID', 'DptDt')

In [ ]:
# save 
wdata.to_csv('4y_prepared_data.csv', sep=';')